# softmax回归从零开始实现
我们首先直接使用d2l中封装的load_data_fashion_mnist构建数据集迭代器，并设置batch_size=256：

In [34]:
import torch
from d2l import torch as d2l
import warnings
import torchvision
from torchvision import transforms
from torch.utils import data
warnings.filterwarnings('ignore')


In [35]:
batch_size = 256
def load_data_fashion_mnist(batch_size, resize=None):
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root='../data', train=True, transform=trans, download= True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="../data", train=False, transform=trans, download=True)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=0),
            data.DataLoader(mnist_test, batch_size, shuffle=True, num_workers=0))
train_iter, test_iter = load_data_fashion_mnist(batch_size)

# 初始化模型参数
和之前线性回归的例子一样，这里的每个样本都将用固定长度的向量表示。
原始数据集中的每个样本都是$28 \times 28$的图像。
在本节中，我们[**将展平每个图像，把它们看作长度为784的向量。**]

在softmax回归中，我们的输出与类别一样多。
(**因为我们的数据集有10个类别，所以网络输出维度为10**)。
因此，权重将构成一个$784 \times 10$的矩阵，
偏置将构成一个$1 \times 10$的行向量。
与线性回归一样，我们将使用正态分布初始化我们的权重`W`，偏置初始化为0。

In [54]:
num_inputs = 784
num_outputs = 10
W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

# 定义softmax操作
在实现softmax回归模型之前，我们先回归一下sum运算符如何沿着张量中的特定维度工作：给定一个矩阵X，我们可以对其所有元素求和（默认情况下），也可以只求同一个轴上的元素之和。

In [37]:
X = torch.tensor([[1.0, 2.0, 3.0], 
                  [4.0, 5.0, 6.0]])
X.sum(), X.sum(axis=0, keepdim=True), X.sum(axis=1, keepdim=True)

(tensor(21.),
 tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

回想一下，[**实现softmax**]由三个步骤组成：

1. 对每个项求幂（使用`exp`）；
1. 对每一行求和（小批量中每个样本是一行），得到每个样本的规范化常数；
1. 将每一行除以其规范化常数，确保结果的和为1。

在查看代码之前，我们回顾一下这个表达式：

(**
$$
\mathrm{softmax}(\mathbf{X})_{ij} = \frac{\exp(\mathbf{X}_{ij})}{\sum_k \exp(\mathbf{X}_{ik})}.
$$
**)

分母或规范化常数，有时也称为*配分函数*（其对数称为对数-配分函数）。


In [38]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(axis=1, keepdim=True)  # 每一行求和，每一行代表一个样本
    return X_exp / partition  # 这里应用了广播机制

下面举一个例子来验证softmax函数的有效性：

In [39]:
X = torch.normal(0, 1, size=(2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(axis=1, keepdim=True)

(tensor([[0.4360, 0.0252, 0.4139, 0.0533, 0.0716],
         [0.2226, 0.4860, 0.0793, 0.1048, 0.1073]]),
 tensor([[1.0000],
         [1.0000]]))

注意，虽然这在数学上看起来是正确的，但我们在代码实现中有点草率。
矩阵中的非常大或非常小的元素可能造成数值上溢或下溢，但我们没有采取措施来防止这点。

## 定义模型

定义softmax操作后，我们可以[**实现softmax回归模型**]。
下面的代码定义了输入如何通过网络映射到输出。
注意，将数据传递到模型之前，我们使用`reshape`函数将每张原始图像展平为向量。

In [40]:
def net(X):
    y = torch.matmul(X.reshape(-1, W.shape[0]), W) + b
    return softmax(y)

# 定义损失函数
我们的输入是预测概率y_hat和原始标签y，假设batch_size，那么y_hat是一个batch_size * num_outputs的矩阵，而y是一个1 * batch_size的向量。我们需要将每个样本标签对应的预测概率拿出来：使用的方法是，将标签作为索引

In [41]:
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.2, 0.3, 0.5]])
y = torch.tensor([0, 2])
y_hat[[0, 1], y], torch.argmax(y_hat, axis=1), y_hat.argmax(axis=1)

(tensor([0.1000, 0.5000]), tensor([2, 2]), tensor([2, 2]))

In [42]:
def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])
cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])

# 分类精度
为了计算精度，我们执行以下操作。
首先，如果`y_hat`是矩阵，那么假定第二个维度存储每个类的预测分数。
我们使用`argmax`获得每行中最大元素的索引来获得预测类别。
然后我们[**将预测类别与真实`y`元素进行比较**]。
由于等式运算符“`==`”对数据类型很敏感，
因此我们将`y_hat`的数据类型转换为与`y`的数据类型一致。
结果是一个包含0（错）和1（对）的张量。
最后，我们求和会得到正确预测的数量。

In [43]:
def accuracy(y_hat, y):
    # 如果y_hat是一个二维矩阵：
    if len(y_hat) > 1 and y_hat.shape[1] > 1:
        y_hat = torch.argmax(y_hat, axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [44]:
accuracy(y_hat, y) / len(y)

0.5

In [45]:
def evaluate_accuracy(net, data_iter):
    """计算指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()
    correct = 0  # 预测正确的样本数量
    samples = 0  # 样本总数量
    with torch.no_grad():
        for X, y in data_iter:
            y_hat = net(X)
            correct += accuracy(y_hat, y)
            samples += len(y)
    return correct / samples

In [46]:
evaluate_accuracy(net, train_iter)

0.12465

In [47]:
def evaluate_accuracy1(net, data_iter):  #@save
    """计算在指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设置为评估模式
    metric = Accumulator(2)  # 正确预测数、预测总数
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [48]:
class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [49]:
evaluate_accuracy1(net, train_iter)

0.12465

# 训练
在这里，我们重构训练过程的实现以使其可重复使用。
首先，我们定义一个函数来训练一个迭代周期。
请注意，`updater`是更新模型参数的常用函数，它接受批量大小作为参数。
它可以是`d2l.sgd`函数，也可以是框架的内置优化函数。


In [57]:
def train_epoch_ch3(net, train_iter, loss, updater):
    if isinstance(net, torch.nn.Module):
        net.train()
    loss_total = 0
    correct = 0
    samples =  0
    for X, y in train_iter:
        y_hat = net(X)
        loss_batch = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # 使用pytorch内置的优化器和损失函数
            updater.zero_grad()
            loss_batch.mean().backward()
            updater.step()
        else:
            # 使用自定义的优化器和损失函数
            loss_batch.sum().backward()
            updater(X.shape[0])
            
        loss_total += loss_batch.sum()
        correct += accuracy(y_hat, y)
        samples += X.shape[0]
    return loss_total / samples, correct / samples

In [51]:
def train_ch3(net, train_iter, test_iter, loss, updater, num_epochs):
    for epoch in range(num_epochs):
        loss_epoch_train, accuracy_epoch_train = train_epoch_ch3(net, train_iter, 
                                                     loss, updater)
        accuracy_epoch_test = evaluate_accuracy(net, test_iter)
        print(f'loss in epoch {epoch + 1}: {loss_epoch_train}, accuracy in epoch {epoch + 1}: {accuracy_epoch_train}')

In [58]:
lr = 0.1

def updater(batch_size):
    return d2l.sgd([W, b], lr, batch_size)

In [59]:
num_epochs = 10
train_ch3(net, train_iter, test_iter, cross_entropy, updater, num_epochs)

loss in epoch 1: 0.5701588988304138, accuracy in epoch 1: 0.8144
loss in epoch 2: 0.5254683494567871, accuracy in epoch 2: 0.82515
loss in epoch 3: 0.5014005303382874, accuracy in epoch 3: 0.8317833333333333
loss in epoch 4: 0.4855089783668518, accuracy in epoch 4: 0.8366166666666667
loss in epoch 5: 0.47400280833244324, accuracy in epoch 5: 0.8402166666666666
loss in epoch 6: 0.46488529443740845, accuracy in epoch 6: 0.8436
loss in epoch 7: 0.4577789902687073, accuracy in epoch 7: 0.8451333333333333
loss in epoch 8: 0.45222848653793335, accuracy in epoch 8: 0.8469833333333333
loss in epoch 9: 0.44759416580200195, accuracy in epoch 9: 0.8482833333333333
loss in epoch 10: 0.44329458475112915, accuracy in epoch 10: 0.8490666666666666
